# Refer to this website to change the Company you want to scrape

https://github.com/robertlandlord/glassdoor_scraper

In [ ]:
#Selenium is an open-source tool that automates web browsers.
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver 
import pandas as pd
from google.colab import drive
import warnings
import traceback
warnings.filterwarnings("ignore")

#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.



from selenium.common.exceptions import NoSuchElementException
from selenium import *
import xlwt
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

     |████████████████████████████████| 904 kB 11.9 MB/s 
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ub

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument("—incognito")
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')
prefs = {'profile.managed_default_content_settings.images':2, 'disk-cache-size': 4096}
option.add_experimental_option("prefs", prefs)

url_link = "https://www.glassdoor.com/Reviews/PwC-Reviews-E8450_P1.htm"
company_info = xlwt.Workbook(encoding="utf-8")
name = input("What would you like to name the file?") + ".csv"
# name = "kpmg.csv"

def scrape(base_url, workbook, name):
    worksheet = workbook.add_sheet("sheet1", cell_overwrite_ok=True)
    worksheet.write(0, 1, "Title")
    worksheet.write(0, 2, "Date Written")
    worksheet.write(0, 3, "Rating")
    worksheet.write(0, 4, "Current/Former")
    worksheet.write(0, 5, "Job Title")
    worksheet.write(0, 6, "Location")
    worksheet.write(0, 7, "Recommendation?")
    worksheet.write(0, 8, "Outlook")
    worksheet.write(0, 9, "Main Text")
    worksheet.write(0, 10, "Pros")
    worksheet.write(0, 11, "Cons")
    # worksheet.write(0, 12, "Advice to management")
    # setting first url
    url = base_url
    chrome_path = 'chromedriver'
    driver = webdriver.Chrome(chrome_path, chrome_options=option)
    # logging in
    driver.get("https://www.glassdoor.com/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK")
    # entering username/password
    username = driver.find_element_by_name("username")
    username.send_keys("mja9@rice.edu")
    password = driver.find_element_by_name("password")
    password.send_keys("vejuW2ga")
    login_btn = driver.find_element_by_xpath("//*[@class='gd-ui-button minWidthBtn css-8i7bc2']")
    driver.execute_script("arguments[0].click();", login_btn)
    
    # after login
    try:
        # go to first link
        driver.get(url)
        url_arr = url.split("_")
        basic = url_arr[0]+"_P"
        page_arr = list(url_arr[1])
        page = ""
        for char in page_arr:
            if char.isdigit():
                page += char
        # this is the current page we are on
        page = int(page)
        counter = 0
        # print(str(url_arr))
        while True:
            # search the company
            # ~~~~~~~~~~~~~~~~~~~TITLE OF REVIEW~~~~~~~~~~~~~~~~~~~~~~
            titles = driver.find_elements_by_class_name("mb-xxsm.mt-0.css-5j5djr")
            # print("titles: " + str(titles))
            titlearr = []
            for title in titles:
                counter += 1
                titlearr.append(title.text)

            # ~~~~~~~~~~~~~~~~~~~~TIMESTAMPS~~~~~~~~~~~~~~~~~~~~~~~
            timestamps = driver.find_elements_by_class_name("authorJobTitle.middle.common__EiReviewDetailsStyle__newGrey")
            # print("timestamps: " + str(timestamps))
            datelist = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            true_timestamps = []
            job_titles = []
            for datetime in timestamps:
                true_timestamps.append(datetime.text.split("-")[0])
                job_titles.append(datetime.text.split("-")[1])



            # ~~~~~~~~~~~~~~~~~~~RATINGS~~~~~~~~~~~~~~~~~~~~~~~~~
            ratings = driver.find_elements_by_class_name("ratingNumber.mr-xsm")
            # print("ratings: " + str(ratings))
            ratingarr = []
            for rating in ratings:  # NEED TO REMOVE THE FIRST ONE BECAUSE ITS GETTING OVERALL RATING
                ratingarr.append(rating.text)
            #if int(page) == 1:
                #del ratingarr[0]

            # ~~~~~~~~~~~~~~~~~~~~CURRENT-FORMER STATUS~~~~~~~~~~~~~~~~~~~~~~~~
            cfstatus = driver.find_elements_by_class_name("pt-xsm.pt-md-0.css-1qxtz39.eg4psks0")
            cfstatusarr = []
            for status in cfstatus:
              cfstatusarr.append(status.text.split(" ")[0])

            # ~~~~~~~~~~~~~~~~~~~~~~~~~FULL REVIEW~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            triples = driver.find_elements_by_xpath('//*[@class="gdReview"]') 
            recarr = []
            outlookarr = []
            othertextarr = []
            prosarr = []
            consarr = []
            advarr = []
            for triple in triples:
                recommends = ""
                outlook = ""
                #ceo = None
                othertext = ""
                pros = ""
                cons = ""
                advice = ""
                textlist = triple.text.splitlines()
                for index in range(0, len(textlist) - 1):
                    if "Recommend" in textlist[index]:
                        recommends = textlist[index]
                    elif "Outlook" in textlist[index]:
                        outlook = textlist[index]
                    #elif "CEO" in textlist[index]:
                    #    ceo = textlist[index]
                    elif "Pros" == textlist[index]:
                        pros = textlist[index + 1]
                        textlist[index + 1] = "got"
                    elif "Cons" == textlist[index]:
                        cons = textlist[index + 1]
                        textlist[index + 1] = "got"
                    elif "Advice to Management" in textlist[index]:
                        advice = textlist[index + 1]
                        textlist[index + 1] = "got"
                    elif "got" == textlist[index]:
                        continue
                    else:
                        othertext = textlist[index]
                recarr.append(recommends)
                outlookarr.append(outlook)
                othertextarr.append(othertext)
                prosarr.append(pros)
                consarr.append(cons)
                advarr.append(advice)

            # print("titlearr: " + str(titlearr))
            # print("true_timestamp: " + str(true_timestamps))
            # print("ratingarr: " + str(ratingarr))
            # print("cfstatusarr: " + str(cfstatusarr))
            for index in range(0, len(titlearr)):
                # print("INDEX IS: ", index)
                # print("ROW: ", int(page)*10+index)
                worksheet.write(int(page)*10+index, 1, titlearr[index])
                worksheet.write(int(page)*10+index, 2, true_timestamps[index])
                worksheet.write(int(page)*10+index, 3, ratingarr[index])
                worksheet.write(int(page)*10+index, 4, cfstatusarr[index])
                worksheet.write(int(page)*10+index, 5, job_titles[index])
                # worksheet.write(int(page)*10+index, 6, locarr[index])
                worksheet.write(int(page)*10+index, 7, recarr[index])
                worksheet.write(int(page)*10+index, 8, outlookarr[index])
                worksheet.write(int(page)*10+index, 9, othertextarr[index])
                worksheet.write(int(page)*10+index, 10, prosarr[index])
                worksheet.write(int(page)*10+index, 11, consarr[index])
                # worksheet.write(int(page)*10+index, 12, advarr[index])
            if page == 200:
              print("Saving workbook...")
              workbook.save(name)
              return
            try:
                found = driver.find_element_by_css_selector("#FooterPageNav > div > ul > li.page.current.last > span")
                print(found.text)
                workbook.save(name)
                break

            # This will probably run every time.
            except NoSuchElementException:
                page += 1
                driver.get(basic+str(page)+".htm")
                print("clicked page", str(page))


    # except StaleElementReferenceException as e:
    #   print(e)
    #   print(traceback.format_exc())
    #   workbook.save(name)
    #   print("finished")

    except Exception as e:
        print(e)
        print(e.args)
        workbook.save(name)
        print("finished")


scrape(url_link, company_info, name)

What would you like to name the file?PwC_Reviews
clicked page 2
clicked page 3
Message: stale element reference: element is not attached to the page document
  (Session info: headless chrome=93.0.4577.63)

('stale element reference: element is not attached to the page document\n  (Session info: headless chrome=93.0.4577.63)', None, None)
finished
